##Pre-processing parameters
</br>seq length threshold</br>count threshold


##Hyperparamters of the model
</br>batch_size
</br>embedding dimensions
</br>LSTM units

##Install and import dependencies

In [1]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.1-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision

In [2]:
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import unicodedata
import re
import time
import keras
import string

print(torch.__version__)

Using TensorFlow backend.


0.4.1


##Load data

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
en = open('/content/drive/My Drive/nmt/europarl-v7.de-en.en', encoding='UTF-8').read().strip().split('\n')
de = open('/content/drive/My Drive/nmt/europarl-v7.de-en.de', encoding='UTF-8').read().strip().split('\n')

##Building vocabulary

###Set vocabulary Hyper-parameters

In [0]:
sent_threshold=15
count_threshold=40

In [0]:
de_m=[]
en_m=[]
for i in range(len(de)):
  if len(de[i].split(' '))< sent_threshold+1 and len(en[i].split(' '))< sent_threshold+1:
    en_m.append(en[i])
    de_m.append(de[i])

###Tokenize

In [0]:
from keras.preprocessing.text import Tokenizer
t= Tokenizer(num_words=30000,lower=True,oov_token='<UNK>')
t.fit_on_texts(en_m)

t_d= Tokenizer(num_words=30000,lower=True,oov_token='<UNK>')
t_d.fit_on_texts(de_m)

In [0]:
t2=Tokenizer()
t_d2=Tokenizer()
t2.word_counts['<UNK>']=0
for word,counts in t.word_counts.items():
  if counts < count_threshold:
    t2.word_index[word]=3
    t2.word_counts['<UNK>']=t2.word_counts['<UNK>']+counts
    
  else:
    t2.word_index[word]=t.word_index[word]+2
    t2.word_counts[word]=t.word_counts[word]
    
t_d2.word_counts['<UNK>']=0
for word,counts in t_d.word_counts.items():
  if counts < count_threshold:
    t_d2.word_index[word]=3
    t_d2.word_counts['<UNK>']=t_d2.word_counts['<UNK>']+counts
    
  else:
    t_d2.word_index[word]=t_d.word_index[word]+2
    t_d2.word_counts[word]=t_d.word_counts[word]

In [0]:
#<s>=1
#</s>=2
#<pad>=0
#UNK=3

In [0]:
t2.word_index['<pad>']=0
t2.word_index['<start>']=1
t2.word_index['<end>']=2

t_d2.word_index['<pad>']=0
t_d2.word_index['<start>']=1
t_d2.word_index['<end>']=2

###debugging....nothing to see here..move on..

In [0]:
a="I'm a gOat wiTh a giant oo'pp!djfkd."
a=keras.preprocessing.text.text_to_word_sequence(a)
table = str.maketrans('', '', string.punctuation)
[w.translate(table) for w in a]

['im', 'a', 'goat', 'with', 'a', 'giant', 'oopp', 'djfkd']

In [15]:
a="I'm a gOat wiTh a giant oo'pp!djfkd."
a=a.split(' ')
table = str.maketrans('', '', string.punctuation)
a=[w.translate(table).lower() for w in a]
a

['im', 'a', 'goat', 'with', 'a', 'giant', 'ooppdjfkd']

##Functions to get padded, indexed sequence

In [0]:
def index_batch(t,batch,flag): #batch will be a list of size [batchsize] with each item being a sentence
  for i in range(len(batch)):
    batch[i]=batch[i].split(' ')
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table).lower() for w in batch[i]]
    for j in range(len(batch[i])):
      if stripped[j]  not in t.word_index:
        batch[i][j]=3
      else:
        batch[i][j]=t.word_index[stripped[j].lower()]
    if flag=='decoder':
      start_token=np.array(1)
      stop_token=np.array(2)
      batch[i]=np.hstack([start_token,batch[i],stop_token])

  return batch

In [0]:
def get_batch(text,t,flag='encoder'):  #returns the padded indexed np.array of given batch number
  batch=index_batch(t,text,flag)
  batch=keras.preprocessing.sequence.pad_sequences(batch,dtype=int,padding='post',truncating='post',value=0)
  return batch

##Parameters

In [0]:
global batch_size, data_size,embedding_dims,units
learning_rate = 0.001
BUFFER_SIZE = len(en_m)
batch_size=32
N_BATCH = BUFFER_SIZE//batch_size
units=128
data_size = len(en_m)
en_vocab_size=len(t2.word_counts)+3 #same as below
de_vocab_size=len(t_d2.word_counts)+3 #adding start stop and pad tokens
embedding_dims=50

##Create dataset

In [0]:
from torch.utils.data import Dataset, DataLoader

In [0]:
class MyData(Dataset):
    def __init__(self, X, y):
        self.data = X
        self.target = y
        # TODO: convert this into torch code is possible
        self.length = [ np.sum(1 - np.equal(x, 0)) for x in X]
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        x_len = self.length[index]
        return x,y,x_len
    
    def __len__(self):
        return len(self.data)

In [0]:
input_tensor = get_batch(en_m,t2,'decoder')
target_tensor = get_batch(de_m,t_d2,'decoder')

In [0]:
train_dataset = MyData(input_tensor, target_tensor)
dataset = DataLoader(train_dataset, batch_size, 
                      drop_last=True,
                      shuffle=True)

##Encoder class

In [0]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.LSTM(self.embedding_dim, self.enc_units)
        
    def forward(self, x, lens, device):
        # x: batch_size, max_length 
        
        # x: batch_size, max_length, embedding_dim
        x=x.transpose(0,1)
        x = self.embedding(x) 
        #print(x.shape)        
        # x transformed = max_len X batch_size X embedding_dim
        x = x.permute(1,0,2)
        #print(x.shape)
        x = pack_padded_sequence(x, lens) # unpad
    
        self.hidden_state = self.initialize_hidden_state(device)
        self.cell_state = self.initialize_hidden_state(device)
        
        # output: max_length, batch_size, enc_units
        # self.hidden: 1, batch_size, enc_units
        output, (self.hidden_state,self.cell_state) = self.gru(x, (self.hidden_state,self.cell_state)) # gru returns hidden state of all timesteps as well as hidden state at last timestep
        
        # pad the sequence to the max length in the batch
        output, _ = pad_packed_sequence(output)
        
        return output, (self.hidden_state, self.cell_state)

    def initialize_hidden_state(self, device):
        return torch.zeros((1, self.batch_sz, self.enc_units)).to(device)

In [0]:
### sort batch function to be able to use with pad_packed_sequence
def sort_batch(X, y, lengths):
    lengths, indx = lengths.sort(dim=0, descending=True)
    X = X[indx]
    y = y[indx]
    return X.transpose(0,1), y, lengths # transpose (batch x seq) to (seq x batch)

In [67]:
### Testing Encoder part
# TODO: put whether GPU is available or not
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
encoder = Encoder(en_vocab_size, embedding_dims, units, batch_size)

encoder.to(device)
# obtain one sample from the data iterator
it = iter(dataset)
x, y, x_len = next(it)

# sort the batch first to be able to use with pac_pack_sequence
xs, ys, lens = sort_batch(x, y, x_len)

enc_output, enc_hidden = encoder(xs.to(device), lens, device)

print(enc_output.size()) # max_length, batch_size, enc_units

torch.Size([17, 32, 128])


##Decoder class

In [0]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dec_units, enc_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.LSTM(self.embedding_dim + self.enc_units, 
                          self.dec_units,
                          batch_first=True)
        self.fc = nn.Linear(self.enc_units, self.vocab_size)
        
        # used for attention
        self.W1 = nn.Linear(self.enc_units, self.dec_units)
        self.W2 = nn.Linear(self.enc_units, self.dec_units)
        self.V = nn.Linear(self.enc_units, 1)
    
    def forward(self, x, hidden,cell, enc_output):
        # enc_output original: (max_length, batch_size, enc_units)
        # enc_output converted == (batch_size, max_length, hidden_size)
        enc_output = enc_output.permute(1,0,2)
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        hidden_with_time_axis = hidden.permute(1, 0, 2)
        
        # score: (batch_size, max_length, hidden_size) # Bahdanaus's
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        # It doesn't matter which FC we pick for each of the inputs
        score = torch.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis))
        
        #score = torch.tanh(self.W2(hidden_with_time_axis) + self.W1(enc_output))
          
        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = torch.softmax(self.V(score), dim=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = torch.sum(context_vector, dim=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        # takes case of the right portion of the model above (illustrated in red)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        #x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        # ? Looks like attention vector in diagram of source
        x = torch.cat((context_vector.unsqueeze(1), x), -1)
        
        # passing the concatenated vector to the GRU
        # output: (batch_size, 1, hidden_size)
        output, (hstate,cstate) = self.gru(x)
        
        
        # output shape == (batch_size * 1, hidden_size)
        output =  output.view(-1, output.size(2))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, (hstate,cstate) ,attention_weights
    
    def initialize_hidden_state(self):
        return torch.zeros((2,1, self.batch_sz, self.dec_units))

In [74]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
encoder = Encoder(en_vocab_size, embedding_dims, units, batch_size)

encoder.to(device)
# obtain one sample from the data iterator
it = iter(dataset)
x, y, x_len = next(it)

print("Input: ", x.shape)
print("Output: ", y.shape)

# sort the batch first to be able to use with pac_pack_sequence
xs, ys, lens = sort_batch(x, y, x_len)

enc_output, (enc_hidden,enc_cell) = encoder(xs.to(device), lens, device)
print("Encoder Output: ", enc_output.shape) # batch_size X max_length X enc_units
print("Encoder Hidden: ", enc_hidden[0].shape) # batch_size X enc_units (corresponds to the last state)

decoder = Decoder(de_vocab_size, embedding_dims, units, units, batch_size)
decoder = decoder.to(device)

#print(enc_hidden.squeeze(0).shape)

dec_hidden = enc_hidden#.squeeze(0)
dec_cell = enc_cell
dec_input = torch.tensor([[t_d2.word_index['<start>']]] * batch_size)
print("Decoder Input: ", dec_input.shape)
print("--------")

for t in range(1, y.size(1)):
    # enc_hidden: 1, batch_size, enc_units
    # output: max_length, batch_size, enc_units
    predictions, (dec_hidden, dec_cell), _ = decoder(dec_input.to(device), 
                                         dec_hidden.to(device),
                                         dec_cell.to(device),
                                         enc_output.to(device))
    
    print("Prediction: ", predictions.shape)
    print("Decoder Hidden: ", dec_hidden.shape)
    
    #loss += loss_function(y[:, t].to(device), predictions.to(device))
    
    dec_input = y[:, t].unsqueeze(1)
    print(dec_input.shape)
    break

Input:  torch.Size([32, 17])
Output:  torch.Size([32, 17])
Encoder Output:  torch.Size([17, 32, 128])
Encoder Hidden:  torch.Size([32, 128])
Decoder Input:  torch.Size([32, 1])
--------
Prediction:  torch.Size([32, 6993])
Decoder Hidden:  torch.Size([1, 32, 128])
torch.Size([32, 1])


##Define loss function

In [0]:
criterion = nn.CrossEntropyLoss()

def loss_function(real, pred):
    """ Only consider non-zero inputs in the loss; mask needed """
    #mask = 1 - np.equal(real, 0) # assign 0 to all above 0 and 1 to all 0s
    #print(mask)
    #print(real)  #[batch_size]
    #print(pred.shape)  #[batch_size x targ_vocab_size]
    #mask = real.ge(1).type(torch.cuda.FloatTensor)
#    print(mask)
    loss_ = criterion(pred, real) 
    return torch.mean(loss_)

In [0]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

## TODO: Combine the encoder and decoder into one class
encoder = Encoder(en_vocab_size, embedding_dims, units, batch_size)
decoder = Decoder(de_vocab_size, embedding_dims, units, units, batch_size)

encoder.to(device)
decoder.to(device)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), 
                       lr=learning_rate)

##Train

In [78]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    encoder.train()
    decoder.train()
    
    total_loss = 0
    
    for (batch, (inp, targ, inp_len)) in enumerate(dataset):
        loss = 0
        
        xs, ys, lens = sort_batch(inp, targ, inp_len)
        
#         print("Output: ", targ.shape)
        enc_output, (enc_hidden,enc_cell) = encoder(xs.to(device), lens, device)
        dec_hidden = enc_hidden
        dec_cell = enc_cell
        
        # use teacher forcing - feeding the target as the next input (via dec_input)
        dec_input = torch.tensor([[t_d2.word_index['<start>']]] * batch_size)
        
        # run code below for every timestep in the ys batch
        for t in range(1, ys.size(1)):
            predictions, (dec_hidden,dec_cell), _ = decoder(dec_input.to(device),
                                                            dec_hidden.to(device),
                                                            dec_cell.to(device),
                                                            enc_output.to(device))
            
            loss += loss_function(ys[:, t].to(device), predictions.to(device))
            #loss += loss_
            dec_input = ys[:, t].unsqueeze(1)
            
        
        batch_loss = (loss / int(ys.size(1)))
        total_loss += batch_loss
        
        optimizer.zero_grad()
        
        loss.backward()

        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.detach().item()))
        
        
    ### TODO: Save checkpoint for model
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    
    
    
    

Epoch 7 Batch 8000 Loss 1.6879
Epoch 7 Batch 8100 Loss 1.2386
Epoch 7 Batch 8200 Loss 1.4558
Epoch 7 Batch 8300 Loss 1.5173
Epoch 7 Batch 8400 Loss 1.9263
Epoch 7 Batch 8500 Loss 1.5180
Epoch 7 Batch 8600 Loss 1.6172
Epoch 7 Batch 8700 Loss 1.5173
Epoch 7 Batch 8800 Loss 1.3046
Epoch 7 Batch 8900 Loss 1.6165
Epoch 7 Batch 9000 Loss 1.6111
Epoch 7 Batch 9100 Loss 1.4874
Epoch 7 Batch 9200 Loss 1.5438
Epoch 7 Batch 9300 Loss 1.9152
Epoch 7 Batch 9400 Loss 1.5763
Epoch 7 Batch 9500 Loss 1.7403
Epoch 7 Batch 9600 Loss 1.2554
Epoch 7 Batch 9700 Loss 1.5551
Epoch 7 Batch 9800 Loss 1.5374
Epoch 7 Batch 9900 Loss 1.5993
Epoch 7 Batch 10000 Loss 1.4041
Epoch 7 Batch 10100 Loss 1.5851
Epoch 7 Batch 10200 Loss 1.3408
Epoch 7 Batch 10300 Loss 1.7249
Epoch 7 Batch 10400 Loss 1.5104
Epoch 7 Batch 10500 Loss 1.5593
Epoch 7 Batch 10600 Loss 1.3964
Epoch 7 Batch 10700 Loss 1.5700
Epoch 7 Batch 10800 Loss 1.5825
Epoch 7 Batch 10900 Loss 1.3377
Epoch 7 Batch 11000 Loss 1.6641
Epoch 7 Batch 11100 Loss 1.4

##Saving learned parameters

In [0]:
torch.save(encoder.state_dict(),'/content/drive/My Drive/nmt/correct_enc_additive_sent15_freq40')

In [0]:
torch.save(decoder.state_dict(),'/content/drive/My Drive/nmt/correct_dec_additive_sent15_freq40')

In [96]:
print("Model's state_dict:")
for param_tensor in decoder.state_dict():
    print(param_tensor, "\t", decoder.state_dict()[param_tensor].size())


Model's state_dict:
embedding.weight 	 torch.Size([6993, 50])
gru.weight_ih_l0 	 torch.Size([512, 178])
gru.weight_hh_l0 	 torch.Size([512, 128])
gru.bias_ih_l0 	 torch.Size([512])
gru.bias_hh_l0 	 torch.Size([512])
fc.weight 	 torch.Size([6993, 128])
fc.bias 	 torch.Size([6993])
W1.weight 	 torch.Size([128, 128])
W1.bias 	 torch.Size([128])
W2.weight 	 torch.Size([128, 128])
W2.bias 	 torch.Size([128])
V.weight 	 torch.Size([1, 128])
V.bias 	 torch.Size([1])


#Loading learned parameters

In [0]:
decoder = Decoder(de_vocab_size, embedding_dims, units, units, batch_size)
decoder = decoder.load_state_dict(torch.load('/content/drive/My Drive/nmt/correct_dec_additive_sent15_freq40'))

In [0]:
dec